# markdown.obsidian.vault

> Functions for inspecting and modifying Obsidian.md vaults and their files and folders 

This module generally manages `Obsidian.md` vaults and their files and folders. 

The `VaultNote` class in this module is one of the most essentially classes used in `trouver`. Generally, one uses `Obsidian.md` to read and write "notes", which are `.md` files. The `VaultNote` class manages such notes in an `Obsidian.md` vault.

See Also `markdown.obsidian.personal.vault`.


In [ ]:
#| default_exp markdown.obsidian.vault

In [ ]:
#| export
from pathlib import Path
import os
from os import PathLike
from typing import Optional, Sequence, Union

from fastcore.basics import patch

from trouver.helper.files_and_folders import (
    path_no_ext, path_name_no_ext
)
from trouver.markdown.obsidian.links import ObsidianLink, LinkType, replace_links_in_text

In [ ]:
import shutil
import tempfile
from unittest import mock

from fastcore.test import *
from nbdev.showdoc import show_doc

from trouver.helper.tests import _test_directory

Some examples for this module come from `nbs/_tests/vault_1`.

## Errors

In [ ]:
#| export
class NoteNotUniqueError(FileNotFoundError):
    """
    A `NoteNotUniqueError` is raised when a `VaultNote` is specified
    by name and not by relative path in the vault, but
    the vault is found to have multiple notes of the name.

    **Attributes**

    - `note_name` - str
        - The name of the note which is not unique in the vault.
    - `notes` - list[str]
        - The paths of the notes whose names are `note_name`.
    """
    def __init__(self, /, *args, **kwargs):
        super().__init__(*args, **kwargs)
            
    @classmethod
    def from_note_names(cls, note_name: str, notes: list[str]):
        """Construct a `NoteNotUniqueError` object from note names"""
        return cls(
            f'The note of the following name is not unique: {note_name}\n'\
            f'The name points to the following files: {notes}')



In [ ]:
show_doc(NoteNotUniqueError.from_note_names)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L41){target="_blank" style="float:right; font-size:smaller"}

### NoteNotUniqueError.from_note_names

>      NoteNotUniqueError.from_note_names (note_name:str, notes:list[str])

*Construct a `NoteNotUniqueError` object from note names*

In [ ]:
#| export
class NoteDoesNotExistError(FileNotFoundError):
    """
    A `NoteDoesNotExistError` is raised when a `VaultNote` is specified
    by either name and or by relative path in the vault, but
    the vault is found to have no notes of the name or path.
    """
    def __init__(self, /, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @classmethod
    def from_note_name(
            cls,
            note_name: str):
        """Construct a `NoteDoesNotExistError` object from note name"""
        return cls(
            f'The note of the following name does not exist: {note_name}.'
            f' Make sure that the argument passed to `note_name` does not'
            f' erroneously end with `.md`, e.g. pass `this_is_a_note`'
            f' instead of `this_is_a_note.md`.')

In [ ]:
show_doc(NoteDoesNotExistError.from_note_name)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L60){target="_blank" style="float:right; font-size:smaller"}

### NoteDoesNotExistError.from_note_name

>      NoteDoesNotExistError.from_note_name (note_name:str)

*Construct a `NoteDoesNotExistError` object from note name*

In [ ]:
#| export
class NoteNotFoundInCacheError(RuntimeError):
    """
    A `NoteNotFoundInCacheError` is raised when a path corresponding to a
    `VaultNote` object is expected to be in the cache, but it is not.
    """
    def __init__(self, /, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @classmethod
    def from_note_name(
            cls,
            note_name: str):
        """Construct a `NoteNotFoundInCacheError` object from note name"""
        return cls(
            f'The note of the following name does not exist: {note_name}.'
            f' Make sure that the argument passed to `note_name` does not'
            f' erroneously end with `.md`, e.g. pass `this_is_a_note`'
            f' instead of `this_is_a_note.md`.')
    

In [ ]:
show_doc(NoteNotFoundInCacheError.from_note_name)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L80){target="_blank" style="float:right; font-size:smaller"}

### NoteNotFoundInCacheError.from_note_name

>      NoteNotFoundInCacheError.from_note_name (note_name:str)

*Construct a `NoteNotFoundInCacheError` object from note name*

In [ ]:
#| export
class NotePathIsNotIdentifiedError(RuntimeError):
    """
    A `NotePathIsNotIdentifiedError` is raised when the `rel_path` attribute of a
    `VaultNote` object is expected to be identified (i.e. a path and not `None`) but
    this expectation is not fulfilled.
    """
    def __init__(self, /, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @classmethod
    def from_note(
            cls,
            note):
        """Construct a `NotePatahIsNotIdentifiedErrro` object from a `VaultNote`."""
        return cls(f'The `rel_path` attribute of the `VaultNote` object is expected'
                   f' to be identified, and yet it is not. The vault of the'
                   f' `VaultNote` object is {note.vault} and the name of the object'
                   f' is {note.name}.')

In [ ]:
show_doc(NotePathIsNotIdentifiedError.from_note)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L102){target="_blank" style="float:right; font-size:smaller"}

### NotePathIsNotIdentifiedError.from_note

>      NotePathIsNotIdentifiedError.from_note (note)

*Construct a `NotePatahIsNotIdentifiedErrro` object from a `VaultNote`.*

## Convert path to Obsidian ID

In [ ]:
#| export
def path_to_obs_id(
        rel_path: PathLike # A path representation the path of an Obsidian note relative to its vault. This does not have to be an existing path.
        ) -> str: # The obsidian url of the hypothetical note within its vault. Note that this does not end with the file extension `.md`.
    """Convert a relative path of an Obsidian note to the Obsidian identifying
    str.
    
    This identification is for a vault-internal Wikilink.

    Note that this function does not have a vault as a parameter.
    """
    path_without_extension = path_no_ext(rel_path)
    return path_without_extension.replace('\\', '/')

Obsidian formats its file paths with '/'; the `path_to_obs_id` function converts a relative path of an Obsidian note to the Obsidian-recognized path.

In [ ]:
test_eq(path_to_obs_id(
    r'some_folder\some_subfolder\some_subsubfolder\some_file.md'),
    'some_folder/some_subfolder/some_subsubfolder/some_file')

Obsidian notes might contain spaces in their paths.

In [ ]:
test_eq(path_to_obs_id(
    Path(r'some folder\some subfolder\some file.md')),
    r'some folder/some subfolder/some file')

## Example vault


## Get all notes

In [ ]:
#| export
def all_paths_to_notes_in_vault(
        vault: PathLike, as_dict: bool = False)\
        -> Union[list[str], dict[str, list[str]]]:
    """Return the paths, relative to the Obsidian vault, of notes 
    in the Obsidian vault.
       
    This may not actually return all of the paths to the notes, see
    the parameter `as_dict`.

    **Parameters**

    - `vault` - `PathLike`
        - The path to the Obsidian vault directory
    - `as_dict` - `bool`
        - If `True`, then returns a dict. If `False`, then returns
        a list. Defaults to `False`. If there are multiple notes with the same
        name in the vault, and `as_dict` is set to `True`, then the dictionary
        will contain only one of the (relative) paths to those notes among its
        values. If `as_dict` is set to `False`, then the list will contain
        all the paths to the notes even when notes with non-unique names exist.
        
    **Returns**

    - Union[list[str], dict[str, str]]
        - Each str represents the path relative to `vault`. If `as_dict` is
        True, then returns a dict whose keys are str, which are (unique) names
        of the notes in the vault, and the values are the paths.
    """
    vault = Path(vault)
    paths =  [os.path.relpath(path, vault) for path in vault.glob(f'**/*.md')]
    if as_dict:
        dicty = {path_name_no_ext(path): [] for path in paths}
        for path in paths:
            dicty[path_name_no_ext(path)].append(path)
        return dicty
    else:
        return paths


The `all_paths_to_notes_in_vault` function returns all of the paths to notes in the Obsidian vault; only `.md` files are recognized as notes.

By default, the function returns a list whose items are strings of paths to notes relative to the vault path.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    # os.startfile(os.getcwd())

    notes = all_paths_to_notes_in_vault(temp_vault)
    test_eq(len(notes), 9)
    print(notes)
    # test_shuffled(, list)
    

['note_2_with_links_to_note_1.md', 'README.md', '_index.md', 'algebra\\ring.md', 'analysis\\exponential_function.md', 'category_theory\\category.md', 'topology\\category.md', 'topology\\note_1.md', 'algebra\\reference_1\\ring.md']


Passing `as_dict=True` returns a dictionary whose keys are note names and whose values are lists of paths to notes of the name relative to the vault.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)
    # os.startfile(os.getcwd())

    notes = all_paths_to_notes_in_vault(temp_vault, as_dict=True)
    test_eq(len(notes), 7)
    test_eq(len(notes['ring']), 2)
    print(notes)


{'note_2_with_links_to_note_1': ['note_2_with_links_to_note_1.md'], 'README': ['README.md'], '_index': ['_index.md'], 'ring': ['algebra\\ring.md', 'algebra\\reference_1\\ring.md'], 'exponential_function': ['analysis\\exponential_function.md'], 'category': ['category_theory\\category.md', 'topology\\category.md'], 'note_1': ['topology\\note_1.md']}


## Searching notes by name

In [ ]:
#| export
def all_note_paths_by_name(
        name: str,  # Name of the note(s) to find
        vault: PathLike,  # The path to the Obsidian vault directory
        subdirectory: Union[PathLike, None] = None # The path to a subdirectory in the Obsidian vault, relative to `vault`. If `None`, then denotes the root of the vault.
        ) -> list[Path]: # Each item is a path to a note of the given name, relative to `vault`.
    """Return the relative paths to all notes in the Obsidian vault 
    with the specified name in the specified subdirectory.
    
    This function does not assume that the specified subdirectory in the vault
    has at most one note of the specified name.
    
    """
    vault = vault if vault != None else ''
    vault = Path(vault)
    subdirectory = subdirectory if subdirectory != None else ''
    directory_path = vault / subdirectory
    all_notes_of_name = directory_path.glob(f'**/{name}.md')
    all_notes_of_name = list(all_notes_of_name)
    return [note_path.relative_to(vault) for note_path in all_notes_of_name]

Basic usage:

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    notes = all_note_paths_by_name('ring', temp_vault)
    print('Searched for notes of name `ring`:')
    print(notes, '\n')
    assert len(notes) == 2

    notes = all_note_paths_by_name('exponential_function', temp_vault)
    print('Searched for notes of name `exponential_function`:')
    print(notes, '\n')
    assert len(notes) == 1

    empty_list = all_note_paths_by_name('non-existent-note-name', temp_vault)
    print('Searching for a non-existent note name yields an empty list:')
    print(empty_list)
    assert len(empty_list) == 0

Searched for notes of name `ring`:
[Path('algebra/ring.md'), Path('algebra/reference_1/ring.md')] 

Searched for notes of name `exponential_function`:
[Path('analysis/exponential_function.md')] 

Searching for a non-existent note name yields an empty list:
[]


We can specify a subdirectory inside the vault to restrict the search to.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    all_notes_named_curve_in_vault = all_note_paths_by_name('category', temp_vault)
    print('All notes named `category`:\n', all_notes_named_curve_in_vault, '\n')
    assert len(all_notes_named_curve_in_vault) == 2
    for note_path in all_notes_named_curve_in_vault:
        test_eq(path_name_no_ext(note_path), 'category')
    
    print('All notes named `category` in the subdirectory `topology`')
    notes_named_topology_in_subdirectory = all_note_paths_by_name(
        'category', temp_vault, 'topology')
    print(notes_named_topology_in_subdirectory)
    assert len(notes_named_topology_in_subdirectory) == 1
    assert path_name_no_ext(notes_named_topology_in_subdirectory[0]) == 'category'

All notes named `category`:
 [Path('category_theory/category.md'), Path('topology/category.md')] 

All notes named `category` in the subdirectory `topology`
[Path('topology/category.md')]


In [ ]:
#| export
# TODO: include examples of `hints` parameter.
def note_path_by_name(
        name: str, # The path to the Obsidian vault directory.
        vault: PathLike, # The path to a subdirectory in the Obsidian vault. If `None`, then denotes the root of the vault.
        subdirectory: Union[PathLike, None] = None, # The path to a subdirectory in the Obsidian vault. If `None`, then denotes the root of the vault.
        hints: Union[list[PathLike], None] = None # Hints of which directories, relative to `subdirectory` that the note may likely be in. This is for speedup. The directories will be searched in the order listed.
        ) -> Path: # The note of the specified name in the specified subdirectory of the vault.
    """Return the path, relative to a subdirectory in the vault, 
    of the note of the specified name.

    **Raises**

    - NoteNotUniqueError
        - If the note of the specified name is not unique in the subdirectory.
    - NoteDoesNotExistError
        - If the note of the specified name does not exist in the subdirectory.

    **See Also**

    - The constructor of the `VaultNote` class
        - passing an argument to the `name` parameter of this constructor
        method essentially does the same thing as this function, except the
        constructor method uses a cache.
    """
    if not subdirectory:
        subdirectory = ''
    if not hints:
        hints = []
    vault = Path(vault)
    subdirectory = Path(subdirectory)
    # absolute_subdirectory = vault / subdirectory
    hints.append('')  # Search in subdirectory if all else fails
    for hint in hints:
        search_results = all_note_paths_by_name(name, vault, subdirectory / hint)
        if len(search_results) > 1:
            raise NoteNotUniqueError.from_note_names(name, search_results)
        elif len(search_results) == 1:
            return search_results[0]
    raise NoteDoesNotExistError.from_note_name(name)

Assuming that there exists a unique note of a specified name in a vault, we can identify it.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    note_path = note_path_by_name('exponential_function', temp_vault)
    print(note_path)
    test_eq(path_name_no_ext(note_path), 'exponential_function')

analysis\exponential_function.md


If there is more than one note in the vault of the specified name, then a `NoteNotUniqueError` is raised.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    print(f'The vault has more than one note named `category`:\n',
            all_note_paths_by_name('category', temp_vault))
    with (ExceptionExpected(ex=NoteNotUniqueError, regex='not unique')):
        note_path_by_name('category', temp_vault)

The vault has more than one note named `category`:
 [Path('category_theory/category.md'), Path('topology/category.md')]


Passing an argument to the parameter `subdirectory` restricts the search to the subdirectory. A `NoteNotUniqueError` can be avoided if a subdirectory is specified and if the note of the specified name is unique in the subdirectory.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    note = note_path_by_name('category', temp_vault, subdirectory='topology')
    print(note)
    assert 'topology' in str(note) and path_name_no_ext(note) == 'category'


topology\category.md


If there is no note in the vault of the specified name, then a `NoteDoesNotExistError` is raised.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    with (ExceptionExpected(ex=NoteDoesNotExistError, regex='does not exist')):
        note_path_by_name('this_note_does_not_exit', temp_vault)

The `hints` parameter

In [ ]:
#| export
def note_name_unique(
        name: str, # Name of the note.
        vault: PathLike # Path to the vault.
        ) -> bool: 
    """Return `True` if a note of the specified name exists and 
    is unique in the Obsidian vault.
    """
    return len(all_note_paths_by_name(name, vault)) == 1

Example use:

In [ ]:
# Example demonstration of the note_name_unique function
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    # Create a temporary vault for testing
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    # Check for a note that is known to exist and is unique
    unique_note_name = 'exponential_function'
    is_unique = note_name_unique(unique_note_name, temp_vault)
    print(f"Is the note '{unique_note_name}' unique in the vault? {is_unique}")
    assert is_unique, f"Expected '{unique_note_name}' to be unique, but it was not."

    # Check for a note that is known to exist but is not unique
    non_unique_note_name = 'category'
    print(f"Checking uniqueness for the note '{non_unique_note_name}'...")
    is_unique = note_name_unique(non_unique_note_name, temp_vault)
    print(f"Is the note '{non_unique_note_name}' unique in the vault? {is_unique}")
    assert not is_unique, f"Expected '{non_unique_note_name}' to not be unique, but it was."

    # Check for a note that does not exist
    nonexistent_note_name = 'this_note_does_not_exist'
    print(f"Checking uniqueness for the note '{nonexistent_note_name}'...")
    is_unique = note_name_unique(nonexistent_note_name, temp_vault)
    print(f"Is the note '{nonexistent_note_name}' unique in the vault? {is_unique}")
    assert not is_unique, f"Expected '{nonexistent_note_name}' to not be unique, but it was."

Is the note 'exponential_function' unique in the vault? True
Checking uniqueness for the note 'category'...
Is the note 'category' unique in the vault? False
Checking uniqueness for the note 'this_note_does_not_exist'...
Is the note 'this_note_does_not_exist' unique in the vault? False


## Getting note name from its path

In [ ]:
#| export
def note_name_from_path(
        note_path: str # The path of the note. The note does not need to exist.
        ) -> str: # The name of the note.
    """Return the name of a note from its path.
    """
    return path_name_no_ext(note_path)

In [ ]:
assert note_name_from_path('algebra/ring.md') == 'ring'

## VaultNote class
Just as how paths in Python can be dealt either via strings of paths or via `pathlib.Path` objects, It is useful to have a class to encapsulate together the name of a note, and its path/Obsidian vault identifier.

In [ ]:
#| export
# TODO: test hidden methods
class VaultNote:
    r"""Represents a note in an Obsidian vault, without regards to the contents.
    
    The note does not have to exist, except in circumstances stating 
    otherwise.

    TODO go through the methods of this class to see which methods assume that
    the note exists and which do not.

    TODO finish the sentence below.
    A `VaultNote` can be specified by either the `rel_path` or the `name` argument
    in its constructor. If `name` is specified, then the 
    
    TODO implement subdirectory hint
    
    **Attributes**

    - vault - Path
        - The (relative or absolute) path of the Obsidian vault
        that the note is located in.
    - name - str
        - The name of the note in the vault.
    - rel_path - str
        - The note's path relative to the vault. If 
    - cache - dict[str, dict[str, list[str]]], class attribute
        - The keys are string, which are paths to vaults. The
        corresponding values are dict whose keys are string, which are
        names in the vault of the (unique) note of that name, and whose
        values are list of string, which are paths to the note relative to the
        vault. The cache is not automatically updated when notes are
        moved, unless the `.move_to` method or its derivatives are invoked.
    
    **Parameters**

    - vault - PathLike
    - rel_path - PathLike
    - name - str
        - The name of the note in the vault. Defaults to the empty str.
            - If `None`, then the `rel_path` parameter should be used 
            to determine `self.name` instead. 
            - If not `None`, then the note must uniquely exist in the
            vault.
    - `subdirectory` - Union[PathLike, None]
    - `hints` - list[PathLike]

    **Raises**
    
    - ValueError
        - if `rel_path` and `name` are both `None`.
    """
    
    cache = {}
    

    @classmethod
    def _check_if_cache_needs_to_update(
            cls, vault: PathLike, name: str) -> bool:
        r"""Returns `True` if the cache needs to update by virtue of not finding
        notes of the specified `name` in the `vault`.
        """
        return not bool(cls._get_from_cache(vault, name))

In [ ]:
#| export
@patch(cls_method=True)
def _get_from_cache(
        cls: VaultNote,
        vault: PathLike,
        name: str,
        ) -> Union[list[str], None]:
    r"""Return the cache's list of notes of the specified name in the
    specified vault.

    If no such list exists in the cache, then return `None`.
    """
    vault = str(vault)
    if vault not in cls.cache:
        return None
    vault_dict = cls.cache[vault]
    if not name in vault_dict:
        return None
    return vault_dict[name]

In [ ]:
#| export
@patch
def _identify_rel_path(
        self: VaultNote
        ) -> Union[Path, None]:
    r"""Returns the Path to the note that this object represents.

    More precisely, if `rel_path` is specified at construction or
    if `self.rel_path` is already identified, then this method
    returns that path. Otherwise, this method looks into the cache to
    check if a note of the `name` specified at construction is in the
    vault and returns the first note in the list of the `name` in the
    cache. If no such note exists, then this method reutrns `None`.
    """
    if self.rel_path is not None:
        return self.rel_path
    cache_search = self.__class__._get_from_cache(self.vault, self.name)
    if cache_search:  # `cache_search` could be `None` or an empty list.
        return cache_search[0]
    return None

@patch
def identify_rel_path(
        self: VaultNote,
        update_cache=False # If `True`, if the cache is searched, and if a note of the specified name is not found in the cache, then the cache is updated and searched again. Defaults to `False`.
        ) -> None:
    r"""Sets `self.rel_path` to a path, if not already done so.

    If `self.rel_path` is not already set as a path, then the cache
    is searched to find a note whose name is `self.name` (which is
    necessarily specified).
    """
    rel_path = self._identify_rel_path()
    if rel_path is not None:
        self.rel_path = rel_path
    elif update_cache:
        self.__class__.update_cache(self.vault)
        self.identify_rel_path(update_cache=False)

In [ ]:
#| export
@patch(cls_method=True)
def update_cache(
        cls: VaultNote,
        vault: PathLike # The vault.
    ) -> None:
    r"""Class method to update cache for `vault` by inspecting all files
    in subdirectories of `vault`."""
    cls.cache[str(vault)] = all_paths_to_notes_in_vault(
        vault, as_dict=True)

        

In [ ]:
#| export
@patch(cls_method=True)
def clear_cache(cls: VaultNote):
    r"""Class method to clear out the entire cache for all vaults."""
    cls.cache = {}
        

In [ ]:
#| export
@patch
def rel_path_identified(self: VaultNote) -> bool:
    r"""Return `True` if `self.rel_path` is identified, i.e. is a path
    that is not `None`."""
    return self.rel_path is not None

In [ ]:
#| export
@patch(cls_method=True)
def _add_single_entry_to_cache(
        cls: VaultNote,
        vault: PathLike,
        rel_path: PathLike) -> None:
    r"""Adds a single entry for a note to the cache.
    
    Does nothing if the entry already exists.

    **Parameters**
    - vault - PathLike
    - rel_path - PathLike
        - The path to the note, relative to `vault`.
    """
    vault_str = str(vault)
    if vault_str not in cls.cache:
        cls.cache[vault_str] = {}
    name = note_name_from_path(rel_path)
    if name not in cls.cache[vault_str]:
        cls.cache[vault_str][name] = []
    if rel_path not in cls.cache[vault_str][name]:
        cls.cache[vault_str][name].append(str(rel_path))

@patch(cls_method=True)
def _remove_single_entry_from_cache(
        cls: VaultNote,
        vault: PathLike,
        rel_path: PathLike) -> None:
    r"""Removes a single entry for a note from the cache.

    Does nothing if the entry is not already there.

    **Parameters**
    - vault - PathLike
    - rel_path - PathLike
        - The path to the note, relative to `vault`.
    """
    vault_str = str(vault)
    if vault_str not in cls.cache:
        return
    name = note_name_from_path(rel_path)
    if name not in cls.cache[vault_str]:
        return
    rel_path = Path(rel_path)
    cls.cache[vault_str][name] = [
        cache_path for cache_path in cls.cache[vault_str][name]
        if Path(cache_path) != rel_path]

## Functions/Methods of the `VaultNote` class

In [ ]:
#| export
# TODO: test/document
@patch
def path(self: VaultNote,
        relative=False # If `True`, then return the path relative to the vault. Otherwise, return the absolute path.
        ) -> Union[Path, None]: # Path to the note if self.rel_path is deterined. `None` otherwise.
    r"""Returns the path to the note.

    **Raises**
    - NotePathIsNotIdentifiedError
        - If the relative path of `self` is not identified.
    """
    if not self.rel_path_identified():
        raise NotePathIsNotIdentifiedError.from_note(self)
    return Path(self.rel_path) if relative\
        else self.vault / self.rel_path

In [ ]:
show_doc(VaultNote.path)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L447){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.path

>      VaultNote.path (relative=False)

*Returns the path to the note.

**Raises**
- NotePathIsNotIdentifiedError
    - If the relative path of `self` is not identified.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| relative | bool | False | If `True`, then return the path relative to the vault. Otherwise, return the absolute path. |
| **Returns** | **Optional** |  | **Path to the note if self.rel_path is deterined. `None` otherwise.** |

In [ ]:
#| export
@patch
# TODO: test/document
def directory(self: VaultNote,
              relative=False # If `True`, then return the path of the directory relative to the vault.
              ) -> Path: # The path of the directory that the note is in.
    r"""Return the directory that the note is in.
    """
    rel_dir = Path(os.path.dirname(self.rel_path))
    return rel_dir if relative else self.vault / rel_dir


In [ ]:
show_doc(VaultNote.directory)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L464){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.directory

>      VaultNote.directory (relative=False)

*Return the directory that the note is in.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| relative | bool | False | If `True`, then return the path of the directory relative to the vault. |
| **Returns** | **Path** |  | **The path of the directory that the note is in.** |

## Constructing VaultNote instances

In [ ]:
#| export
@patch
def __init__(
        self: VaultNote,
        vault: PathLike, # The (relative or absolute) path of the Obsidian vault that the note is located in.
        rel_path: PathLike = None, # The note's path relative to the vault. If `None`, then the `name` parameter is used to determine the note instead. Defaults to `None`.
        name: str = None, # The name of the note. If `None`, then the `rel_path` parameter is used to determine the note instead. Defaults to `None` 
        subdirectory: Union[PathLike, None] = '', # The relative path to a subdirectory in the Obsidian vault. If `None`, then denotes the root of the vault. Defaults to the empty str. 
        hints: list[PathLike] = [], # Paths, relative to `subdirectory`, to directories where the note file may be found. This is for speedup. Defaults to the empty list, in which case the vault note is searched in all of `subdirectory`.
        update_cache: Optional[bool] = True # If `True` and if `rel_path` is not specified (and hence `name` is specified), update the cache
        ):
    # TODO: consider using _check_name_exists_and_unique_in_vault_cache method.
    self.vault = Path(vault)
    if rel_path is None and name is None:
        raise ValueError(
            "In constructing a `VaultNote` object, the parameters `rel_path`"
            " and `name` parameters were expected to be given arguments, but"
            " both parameters are given `None` as arguments.")
    if rel_path is not None:
        self.rel_path = str(rel_path)
        self.name = note_name_from_path(self.rel_path)
    else:
        self.name = name
        self.rel_path = None
        self.identify_rel_path(update_cache=update_cache)

#### By name

The most convenient way to construct an existing `VaultNote` is to specify the vault in which it exists and the note's name, assuming that the note of the specified name exists and is unique in the specified vault. We can also verify the existence of the file corresponding to a `VaultNote` objet with the `.exists` method.

In [ ]:
#| export
@patch
def exists(
        self: VaultNote,
        update_cache=False # If `True`, then update the cache and try to identify `self.rel_path` before verifying whether the note exists in the vault.
        ) -> bool:
    r"""Returns `True` if `self.rel_path` is identified and
    if the note exists in the vault.
    
    Setting `update_cache` to `True` updates the cache before verifying
    whether the `VaultNote` object exists if the `VaultNote` object is
    specified by `name` and not `rel_path`. Doing so guarantees that the
    output is correct at the possible cost of runtime.
    """
    if self.rel_path is None:
        if update_cache:
            self.identify_rel_path(update_cache=True)
        else:
            return False
    try:
        abs_path = self.path()        
    except NotePathIsNotIdentifiedError as e:
        return False
    return os.path.exists(abs_path)

In [ ]:
show_doc(VaultNote.exists)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L501){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.exists

>      VaultNote.exists (update_cache=False)

*Returns `True` if `self.rel_path` is identified and
if the note exists in the vault.

Setting `update_cache` to `True` updates the cache before verifying
whether the `VaultNote` object exists if the `VaultNote` object is
specified by `name` and not `rel_path`. Doing so guarantees that the
output is correct at the possible cost of runtime.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| update_cache | bool | False | If `True`, then update the cache and try to identify `self.rel_path` before verifying whether the note exists in the vault. |
| **Returns** | **bool** |  |  |

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    vault_note = VaultNote(temp_vault, name='exponential_function')
    assert vault_note.exists()
    print(vault_note.name)
    print(f'`vault_note` is located, relative to `vault`, at {vault_note.rel_path}.')

exponential_function
`vault_note` is located, relative to `vault`, at analysis\exponential_function.md.


#### By relative path

Alternatively, a `VaultNote` object can be created by passing an argument to the `rel_path` parameter. In this case, the note of the specified path does not need to exist.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    vault_note = VaultNote(temp_vault, rel_path='non_existent_folder/non_existent_note.md')
    assert not vault_note.exists()
    # Note that there is not a unique note of name `ring`.
    vault_note = VaultNote(temp_vault, rel_path='algebra/ring.md')  
    assert vault_note.exists()

The `rel_path` parameter takes precedence over the `name` parameter.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    vault_note = VaultNote(temp_vault, rel_path='non_existent_folder/non_existent_note.md', name='ring')
    assert vault_note.name == 'non_existent_note'

If the arguments for both the `name` and the `rel_path` parameters are `None`, then a `ValueError` is raised:



In [ ]:
test_vault = _test_directory() / 'test_vault_1'
with ExceptionExpected(ValueError):
    vault_note = VaultNote(test_vault, rel_path=None, name=None)

#### Cache of the `VaultNote` class

In [ ]:
show_doc(VaultNote.update_cache)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L370){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.update_cache

>      VaultNote.update_cache (vault:os.PathLike)

*Class method to update cache for `vault` by inspecting all files
in subdirectories of `vault`.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| vault | PathLike | The vault. |
| **Returns** | **None** |  |

In [ ]:
show_doc(VaultNote.clear_cache)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L383){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.clear_cache

>      VaultNote.clear_cache ()

*Class method to clear out the entire cache for all vaults.*

The `VaultNote` class keeps a cache of the notes (files with extension `.md`) in the vault. In pracctice, this cache is updated when a note of the specified name is not found when constructing a `VaultNote` instance via the `name` parameter.

In [ ]:
#| export
@patch(cls_method=True)
def _check_name_exists_and_unique_in_vault_cache(
        cls: VaultNote,
        vault: PathLike, # The vault
        name: str # The name
        ) -> None:
    r"""
    Raise `NoteNotUniqueError` or `NoteNotFoundInCacheError` if the note of
    the specified name is not unique or is not found in the cache.

    Note that a note may exist but not be found in the cache if it was
    created without using the `VaultNote.create` method. For example,
    this could happen if a note is created by the user using the
    file explorer.
    
    **Parameters**
    - vault - PathLike
    - name - str

    **Raises**
    - `NoteNotUniqueError`
    - `NoteNotFoundInCacheError`.
    """
    vault_dict = cls.cache[str(vault)]
    if not name in vault_dict or len(vault_dict[name]) == 0:
        raise NoteNotFoundInCacheError.from_note_name(name)
    elif len(vault_dict[name]) > 1:
        raise NoteNotUniqueError.from_note_names(
            name, vault_dict[name])
    else: #name in vault_dict and len(vault_dict[name]) == 1
        note_path = vault_dict[name][0]
        if not os.path.exists(Path(vault) / note_path):
            raise NoteDoesNotExistError.from_note_name(name)


In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    VaultNote.update_cache(temp_vault)
    
    name_of_unique_note = 'exponential_function'
    assert VaultNote._check_name_exists_and_unique_in_vault_cache(temp_vault, name_of_unique_note) is None

    with ExceptionExpected(ex=NoteNotFoundInCacheError):
        non_existent_note_name = 'this_note_does_not_exist'
        VaultNote._check_name_exists_and_unique_in_vault_cache(temp_vault, non_existent_note_name)

    with ExceptionExpected(ex=NoteNotFoundInCacheError):
        # The following note file is created without using the `VaultNote.cretae` method.
        # As such, the cache is not updated with this new note.
        to_create_note_name = 'note_created_without_using_VaultNote_create'
        open(temp_vault / f'{to_create_note_name}.md', 'w').close()
        VaultNote._check_name_exists_and_unique_in_vault_cache(temp_vault, to_create_note_name)

    with ExceptionExpected(ex=NoteDoesNotExistError):
        # The following note file is deleted (without updating the cache).
        # As such, while the cache indicates that
        # the file may exist, The `VaultNote._check_name_exists_and_unique_in_vault_cache`
        # actually checks whether the file exists and concludes that it does not.
        to_delete_note_name = 'note_1'
        to_delete_note_rel_path = f'topology/{to_delete_note_name}.md'
        os.remove(temp_vault / to_delete_note_rel_path)
        VaultNote._check_name_exists_and_unique_in_vault_cache(temp_vault, to_delete_note_name)

    with ExceptionExpected(ex=NoteNotUniqueError):
        existent_but_non_unique_note_name = 'ring'
        VaultNote._check_name_exists_and_unique_in_vault_cache(temp_vault, existent_but_non_unique_note_name)

    with ExceptionExpected(ex=NoteNotUniqueError):
        # In this example, the note name is not unique. 
        # The note file is also deleted without updating the cache.
        # Nevertheless, the `VaultNote._check_name_exists_and_unique_in_vault_cache`
        # determines that the note name is not unique.
        to_delete_note_name = 'category'
        to_delete_note_rel_path = f'category_theory/{to_delete_note_name}.md'
        os.remove(temp_vault / to_delete_note_rel_path)
        VaultNote._check_name_exists_and_unique_in_vault_cache(temp_vault, to_delete_note_name)

In [ ]:
VaultNote.clear_cache()
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    assert VaultNote._check_if_cache_needs_to_update(temp_vault, name='ring')
    VaultNote.update_cache(temp_vault)
    assert VaultNote._check_if_cache_needs_to_update(temp_vault, name='does_not_exist')

#### Identifying the `VaultNote` object

In [ ]:
show_doc(VaultNote.identify_rel_path)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L351){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.identify_rel_path

>      VaultNote.identify_rel_path (update_cache=False)

*Sets `self.rel_path` to a path, if not already done so.

If `self.rel_path` is not already set as a path, then the cache
is searched to find a note whose name is `self.name` (which is
necessarily specified).*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| update_cache | bool | False | If `True`, if the cache is searched, and if a note of the specified name is not found in the cache, then the cache is updated and searched again. Defaults to `False`. |
| **Returns** | **None** |  |  |

In [ ]:
show_doc(VaultNote.rel_path_identified)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L390){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.rel_path_identified

>      VaultNote.rel_path_identified ()

*Return `True` if `self.rel_path` is identified, i.e. is a path
that is not `None`.*

When the `name` parameter (as opposed to the `rel_path` parameter) is specified in the constructor of a `VaultNote` object, the constructor looks into the cache of the `VaultNote` class to identify a note with the specified name in the specified vault. If the cache contains no such note, then the cache is updated and searched again. If the cache still contains no such note, then the `rel_path` attribute of the `VaultNote` object is left unidentified (i.e. is set to `None`). 

Assuming that a note of the specified name is created later, the relative path of the `VaultNote` object can be identified using the `identify_rel_path(update_cache=True)` method. Moreover, the `rel_path_identified` method returns `True` if the relative path is identified.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    vn = VaultNote(temp_vault, name='does_not_exist_at_first')
    assert not vn.rel_path_identified()
    # Create a note at the root of the vault.
    open(temp_vault / 'does_not_exist_at_first.md', 'w').close()
    vn.identify_rel_path(update_cache=True)
    assert vn.rel_path_identified()
    assert vn.exists()
    test_eq(vn.rel_path, 'does_not_exist_at_first.md')
    

## Getting information about the note

In [ ]:
#| export
@patch
def obsidian_identifier(self: VaultNote) -> str:
    r"""Return the Obsidian identifier of the `VaultNote` object.
    
    This is the note's unqiue Obsidian id in the vault. This is like a
    path str with forward slashes `/` (as opposed to backwards `\` slashes)
    and without a file extension (`.md`).
    """
    return path_to_obs_id(self.rel_path)

In [ ]:
show_doc(VaultNote.obsidian_identifier)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L562){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.obsidian_identifier

>      VaultNote.obsidian_identifier ()

*Return the Obsidian identifier of the `VaultNote` object.

This is the note's unqiue Obsidian id in the vault. This is like a
path str with forward slashes `/` (as opposed to backwards `\` slashes)
and without a file extension (`.md`).*

Here are some convenient ways to get information about the `VaultNote`:

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    vault_note = VaultNote(temp_vault, name='exponential_function')
    print(f'Obsidian vault identifier:\t{vault_note.obsidian_identifier()}')
    print(f'relative path:\t{vault_note.rel_path}')
    print(f'Part of the absolute path of the note:\t{str(vault_note.path(relative=False))[:7]}')
    print(f'note name:\t{vault_note.name}')
    print(f'directory that the note is in relative to the vault:\t{vault_note.directory(relative=True)}')

Obsidian vault identifier:	analysis/exponential_function
relative path:	analysis\exponential_function.md
Part of the absolute path of the note:	c:\User
note name:	exponential_function
directory that the note is in relative to the vault:	analysis


The `VaultNote.path` method raises a `NotePathIsNotIdentifiedError` if the `VaultNote` object's relative path is not idetnfied:

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    vn = VaultNote(temp_vault, name='does_not_exist')
    with ExceptionExpected(NotePathIsNotIdentifiedError):
        vn.path()

## Reading the contents of the note

The `VaultNote.text()` function reads the contents of the file that the `VaultNote` object represents, assuming that this file exists. If the file does not exist, then a `NoteDoesNotExistError` is raised.

In [ ]:
#| export
@patch
def text(
        self: VaultNote
        ) -> str: # Text contained in the note, assuming that the note exists.
    r"""Returns the text contained in the note.
    
    **Raises**

    - NoteDoesNotExistError
        - If `self` does not point to an existing note.
    """
    if not self.exists():
        raise NoteDoesNotExistError.from_note_name(self.name)
    with open(self.vault / self.rel_path, 
                'r', encoding='utf-8') as reader:
        text = reader.read()
        reader.close()
    return text


In [ ]:
show_doc(VaultNote.text)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L573){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.text

>      VaultNote.text ()

*Returns the text contained in the note.

**Raises**

- NoteDoesNotExistError
    - If `self` does not point to an existing note.*

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)
    vault_note = VaultNote(temp_vault, name='exponential_function')
    print(vault_note.text())
    assert len(vault_note.text()) > 10

    vault_note = VaultNote(temp_vault, rel_path='does_not_exist.md')
    with ExceptionExpected(ex=NoteDoesNotExistError):
        vault_note.text()

The **exponential function** is the function sending a complex number $z$ to **$$e^{z} = \sum_{n=0}^\infty \frac{z^n}{n!}$$**. It converges for all $z \in \mathbb{C}$.


## Creating/deleting/moving the note

If a `VaultNote` object represents a non-existent file, then the file can be created with empty content. The cache is updated with this single new entry, but the rest of the cache remains the same.

If the file exists, then a `FileExistsError` is raised.

If the specified directory for the file does not exist, then a `FileNotFoundError` is raised.

In [ ]:
#| export
@patch
def create(self: VaultNote):
    # TODO: consider using _check_name_exists_and_unique_in_vault_cache method.
    r"""Create the note if it does not exist.
    
    The directory of the file needs to be created separately
    beforehand.

    If the file exists, then a FileExistsError is raised and
    the file modification time is not changed.
    
    **Raises**

    - FileExistsError
        - If the file already exists.
    - FileNotFoundError
        - If the directory of the file does not already exist.
    """
    Path(self.path()).touch(exist_ok=False)
    self.__class__._add_single_entry_to_cache(
        self.vault, self.rel_path)


In [ ]:
show_doc(VaultNote.create)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L594){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.create

>      VaultNote.create ()

*Create the note if it does not exist.

The directory of the file needs to be created separately
beforehand.

If the file exists, then a FileExistsError is raised and
the file modification time is not changed.

**Raises**

- FileExistsError
    - If the file already exists.
- FileNotFoundError
    - If the directory of the file does not already exist.*

In [ ]:
#| export
@patch
def delete(self: VaultNote):
    r"""Delete the note if it exists.
    
    This updates the cache if necessary
    """
    if self.exists(update_cache=True):
        os.remove(self.path())
        self.__class__._remove_single_entry_from_cache(self.vault, self.rel_path)

In [ ]:
show_doc(VaultNote.delete)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L618){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.delete

>      VaultNote.delete ()

*Delete the note if it exists.

This updates the cache if necessary*

In [ ]:
#| export
@patch
def move_to(self: VaultNote,
            rel_path: PathLike # The path in which to rename the path to `self` as, relative to `self.vault`.
            ) -> None:
    r"""Move/rename the note to the specified location in the vault,
    assuming that it exists.

    Nothing is done if the note does not exist.
    """
    if not self.exists():
        return
    os.rename(self.path(), Path(self.vault) / rel_path)
    self.__class__._remove_single_entry_from_cache(
        self.vault, Path(self.rel_path))
    self.__class__._add_single_entry_to_cache(
        self.vault, Path(rel_path))
    self.rel_path = str(rel_path)
    self.name = note_name_from_path(self.rel_path)
    
@patch
def move_to_folder(self: VaultNote,
                    rel_dir: PathLike # The path of the directory in which to move `self` to, relative to `self.vault`.
                    ) -> None:
    # TODO: consider using _check_name_exists_and_unique_in_vault_cache method.
    r"""Move the note to the specified folder in the vault, assuming that
    it exists.
    """
    self.move_to(Path(rel_dir) / f'{self.name}.md')

In [ ]:
show_doc(VaultNote.move_to)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L629){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.move_to

>      VaultNote.move_to (rel_path:os.PathLike)

*Move/rename the note to the specified location in the vault,
assuming that it exists.

Nothing is done if the note does not exist.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| rel_path | PathLike | The path in which to rename the path to `self` as, relative to `self.vault`. |
| **Returns** | **None** |  |

In [ ]:
show_doc(VaultNote.move_to_folder)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L648){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.move_to_folder

>      VaultNote.move_to_folder (rel_dir:os.PathLike)

*Move the note to the specified folder in the vault, assuming that
it exists.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| rel_dir | PathLike | The path of the directory in which to move `self` to, relative to `self.vault`. |
| **Returns** | **None** |  |

In [ ]:
#| export
@patch
def rename(
        self: VaultNote,
        new_name: str,  # The new name to give the note('s file). Should not include the `.md` extension.`
        replace_links_in_vault: bool = True  # If `True`, then replace the links in the vault pointing to `note` to reflect the new name.
        ):

    # TODO: consider using _check_name_exists_and_unique_in_vault_cache method.
    r"""
    Rename the file underlying `self` to `new_name`. The directory that the file is in remains unchanged.

    Assumes that
    
    1. the name of `self` is unique among note names in `self.vault`s
    2. No pre-existing note in `vault` has `new_name` as its name. Use the `note_name_unique`
        function to check whether or not this is the case.
    3. the class' `.cache` accurately reflects the files in `vault`

    """
    parent_dir = Path(os.path.dirname(self.rel_path))
    old_wikilink_pattern_by_name = ObsidianLink(
        is_embedded=False, file_name=self.name, anchor=-1, custom_text=-1)
    old_markdownlink_pattern_by_name_1 = ObsidianLink(
        is_embedded=False, file_name=f'{self.name}.md', anchor=-1, custom_text=-1, link_type=LinkType.MARKDOWN)
    old_markdownlink_pattern_by_name_2 = ObsidianLink(
        is_embedded=False, file_name=f'{self.name}', anchor=-1, custom_text=-1, link_type=LinkType.MARKDOWN)
    self.move_to(parent_dir / f'{new_name}.md')
    if not replace_links_in_vault:
        return
    for name, paths in self.__class__.cache[str(self.vault)].items():
        for path in paths:
            other_note = VaultNote(vault=self.vault, rel_path=path)
            if not other_note.exists():
                continue
            original_text = other_note.text()
            text = original_text
            text = replace_links_in_text(text, old_wikilink_pattern_by_name, new_link_name=new_name)
            text = replace_links_in_text(text, old_markdownlink_pattern_by_name_1, new_link_name=f'{new_name}.md')
            text = replace_links_in_text(text, old_markdownlink_pattern_by_name_2, new_link_name=f'{new_name}')
            if text != original_text:
                with open(other_note.path(), 'w', encoding='utf-8') as file:
                    file.write(text)
                    file.close()

In [ ]:
show_doc(VaultNote.rename)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L659){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.rename

>      VaultNote.rename (new_name:str, replace_links_in_vault:bool=True)

*Rename the file underlying `self` to `new_name`. The directory that the file is in remains unchanged.

Assumes that

1. the name of `self` is unique among note names in `self.vault`s
2. No pre-existing note in `vault` has `new_name` as its name. Use the `note_name_unique`
    function to check whether or not this is the case.
3. the class' `.cache` accurately reflects the files in `vault`*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| new_name | str |  | The new name to give the note('s file). Should not include the `.md` extension.` |
| replace_links_in_vault | bool | True | If `True`, then replace the links in the vault pointing to `note` to reflect the new name. |

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)
    vault_note = VaultNote(temp_vault, rel_path='new_file.md')
    vault_note.create()
    assert vault_note.exists()
    assert vault_note.rel_path in VaultNote.cache[str(temp_vault)]['new_file']

    with ExceptionExpected(ex=FileExistsError):
        vault_note.create() 

    vault_note = VaultNote(temp_vault, rel_path='none_existent_folder/new_file.md')
    with ExceptionExpected(ex=FileNotFoundError):
        vault_note.create()

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)
    vault_note = VaultNote(temp_vault, name='exponential_function')
    vault_note.delete()
    assert not vault_note.exists()
    assert vault_note.rel_path not in VaultNote.cache[str(temp_vault)]['exponential_function']

If a `VaultNote` object represents an existing file, then the file can be renamed or moved.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)

    VaultNote.clear_cache()
    vault_note = VaultNote(temp_vault, name='exponential_function')
    vault_note.move_to_folder('')
    assert vault_note.rel_path == 'exponential_function.md'
    assert 'exponential_function.md' in VaultNote.cache[str(temp_vault)]['exponential_function']

The `VaultNote.rename` function renames the file underlying an existing vault note.

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)
    vault_note = VaultNote(temp_vault, name='exponential_function')
    print(f"Old path: {vault_note.rel_path}")
    vault_note.rename('exp_func')

    expected_path = os.path.normpath('analysis/exp_func.md')
    actual_path = os.path.normpath(vault_note.rel_path)
    test_eq(actual_path, expected_path)

    # test_eq(Path(vault_note.rel_path), Path(r'analysis\exp_func.md'))
    assert vault_note.exists()
    print(f"New path: {vault_note.rel_path}")

Old path: analysis\exponential_function.md
New path: analysis\exp_func.md


By default, the `rename` method replaces links in notes in the vault 

In [ ]:

with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)
    vault_note = VaultNote(temp_vault, name='note_1')
    other_note = VaultNote(temp_vault, name='note_2_with_links_to_note_1')
    print(f"Old path: {vault_note.rel_path}")
    print(f"`note_2_with_links_to_note_1` is a note with links to `note_1`. The following is its contents:")
    print(other_note.text())
    vault_note.rename('renamed_note_1')

    assert other_note.exists()
    print(f"\nNew path: {vault_note.rel_path}")
    print(f"After `note_1` is renamed, the following is the contents of `note_2_with_links_to_note_1`:")

    new_text_in_note_2 = other_note.text()
    print(new_text_in_note_2)
    assert '[[renamed_note_1]]' in new_text_in_note_2
    assert '[[renamed_note_1|hi]]' in new_text_in_note_2
    assert '[[renamed_note_1#blahblah|hi]]' in new_text_in_note_2
    assert '![[renamed_note_1|embedded]]' in new_text_in_note_2
    assert '[asdf](renamed_note_1)' in new_text_in_note_2
    assert '[asdf](renamed_note_1.md)' in new_text_in_note_2

Old path: topology\note_1.md
`note_2_with_links_to_note_1` is a note with links to `note_1`. The following is its contents:
[[note_1]]

[[note_1|hi]]
[[note_1#blahblah|hi]]

![[note_1|embedded]]

[asdf](note_1)

[asdf](note_1.md)

New path: topology\renamed_note_1.md
After `note_1` is renamed, the following is the contents of `note_2_with_links_to_note_1`:
[[renamed_note_1]]

[[renamed_note_1|hi]]
[[renamed_note_1#blahblah|hi]]

![[renamed_note_1|embedded]]

[asdf](renamed_note_1)

[asdf](renamed_note_1.md)


### Getting a unique note name

It is troublesome to create a note with a non-unique name. The `unique_name` method of the VaultNote class takes a tentative name for a note to be created and adds a number to the name so that the note name will be unique in the vault. 

In [ ]:
#| export

@patch(cls_method=True)
def unique_name(
        cls: VaultNote,
        name: str, # The base name for the note.
        vault: PathLike, # The vault
        other_unavailable_names: Optional[Sequence[str]] = None # Other names that should be excluded when generating the unique name
        ) -> str: # A str obtained by appending `_{some number}` to the end of `name`.
    r"""A class method to return a name for a note that is unique in
    the vault based on a specified name.
    """
    if not str(vault) in VaultNote.cache:
        cls.update_cache(vault)
    unavailable_names = set(VaultNote.cache[str(vault)])
    if other_unavailable_names is None:
        other_unavailable_names = []
    unavailable_names.update(other_unavailable_names)
    if not name in unavailable_names:
    # if not name in VaultNote.cache[str(vault)]:
        return name
    i = 1
    # while f'{name}_{i}' in VaultNote.cache[str(vault)]:
    while f'{name}_{i}' in unavailable_names:
        i += 1
    return f'{name}_{i}'

In [ ]:
show_doc(VaultNote.unique_name)

---

[source](https://github.com/hyunjongkimmath/trouver/blob/main/trouver/markdown/obsidian/vault.py#L704){target="_blank" style="float:right; font-size:smaller"}

### VaultNote.unique_name

>      VaultNote.unique_name (name:str, vault:os.PathLike,
>                             other_unavailable_names:Optional[Sequence[str]]=No
>                             ne)

*A class method to return a name for a note that is unique in
the vault based on a specified name.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| name | str |  | The base name for the note. |
| vault | PathLike |  | The vault |
| other_unavailable_names | Optional | None | Other names that should be excluded when generating the unique name |
| **Returns** | **str** |  | **A str obtained by appending `_{some number}` to the end of `name`.** |

In [ ]:
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)
    # There is a note named category in the test vault.
    sample_name = VaultNote.unique_name('category', temp_vault)  
    assert not VaultNote(temp_vault, name=sample_name).exists()

    sample_name = VaultNote.unique_name('non_existent_note_name', temp_vault)
    assert not VaultNote(temp_vault, name=sample_name).exists()
    assert sample_name == 'non_existent_note_name'

    sample_name = VaultNote.unique_name('category', temp_vault, other_unavailable_names=['category_1']) 
    assert not VaultNote(temp_vault, name=sample_name).exists()
    assert sample_name not in ['category', 'category_1']

category_2


In [ ]:
#| hide
# This import 
with tempfile.TemporaryDirectory(prefix='temp_dir', dir=os.getcwd()) as temp_dir:
    temp_vault = Path(temp_dir) / 'test_vault_1'
    shutil.copytree(_test_directory() / 'test_vault_1', temp_vault)
    # Test _add_single_entry_to_cache
    VaultNote.clear_cache()
    VaultNote._add_single_entry_to_cache(temp_vault, rel_path='directory/file.md')
    assert len(VaultNote.cache) == 1
    assert len(VaultNote.cache[str(temp_vault)]) == 1
    assert len(VaultNote.cache[str(temp_vault)]['file']) == 1

    VaultNote._remove_single_entry_from_cache(temp_vault, rel_path=Path('directory/file.md'))
    assert len(VaultNote.cache) == 1
    assert len(VaultNote.cache[str(temp_vault)]) == 1
    assert len(VaultNote.cache[str(temp_vault)]['file']) == 0
    # Test update_cache with mock
    with mock.patch('__main__.all_paths_to_notes_in_vault') as mock_method:
        VaultNote.update_cache(temp_vault)
        mock_method.assert_called()

    # with (mock.patch.object(VaultNote, '_add_single_entry_to_cache') as mock_add_cache,
    #       mock.patch.object(VaultNote, '_remove_single_entry_from_cache') as mock_remove_cache,
    #       ):

    # See https://stackoverflow.com/questions/29152170/what-is-the-difference-between-mock-patch-object-and-mock-patch
    with (mock.patch('__main__.VaultNote._add_single_entry_to_cache') as mock_add_cache,
          mock.patch('__main__.VaultNote._remove_single_entry_from_cache') as mock_remove_cache,
          ):
        vault_note = VaultNote(temp_vault, name='exponential_function')
        vault_note.move_to_folder('')
        # mock_add_cache.assert_called_with()
        mock_remove_cache.assert_called_with(temp_vault, Path('analysis') / Path('exponential_function.md'))
        mock_add_cache.assert_called_with(temp_vault, Path('exponential_function.md'))

In [ ]:
## Copying files in an `Obsidian.md` vault to and from a subvault

In [ ]:
# # TODO: use these methods during vault construction for a reference
# def copy_vault_file_into_subvault(
#         vault: PathLike, # The Path to the vault from which to copy the files.
#         subvaults: Union[PathLike, list[PathLike]], # The Paths to the subvaults to which to copy the files.
#         files: Union[PathLike, list[PathLike]], # The Path to the files, relative to `vault` to copy.
#         replace: bool = True, # If `True`, replace existing files in `subvaults` if necessary. Defaults to `True`
#         backup: bool = True, # If `True` and if `replace=True`, create a backup for any replaced files in a subvault in a folder named `.back` in the root directory of the subvault.
#         ) -> None: 
#     """Copy the specified files in `vault` into subvaults.

#     The files are copied within the subvaults to the same relative paths as
#     they are found in `vault`.

#     Here, "files" include directories. If a directory is copied, then all
#     files and subdirectories of that directory are also copied.

#     **Parameters**
#     - vault - PathLike
#         - The path to the Obsidian vault from which to copy files from.
#     - subvaults - PathLike or list[PathLike]
#         - The paths to the subvaults to which to copy the files.
#     - files - PathLike or list[PathLike]
#         - The files to copy.

#     **Raises**
#     - FileExistsError
#         - If `replace` is `False` and some subvault already has a file at
#           the path in which a file-copy is attempted. In this case, no
#           files are copied.
#     - FileNotFoundError
#         - If a path specified in `files` does not exist in `vault`. In this
#           case, no files are copied.

#     """
#     vault = Path(vault)
#     if isinstance(subvaults, PathLike):
#         subvaults = [subvaults]
#     if isinstance(files, PathLike):

#         files = [files]


#     # TODO: Implement this as a private function 
#     for file in files:
#         if not os.path.exists(vault / file):
#             raise FileNotFoundError(
#                 f"Attempted to copy files/folders from a vault into subvaults"
#                 f", but there is at least one non-existent files"
#                 f". No files have been copied."
#                 f"vault: {vault}"
#                 f"file: {file}")

#     if not replace: 
#         for subvault, file in itertools.product(subvaults, files):
#             if os.path.exists(subvault / file):
#                 raise FileExistsError(
#                     f"Attempted to copy files/folders from a vault into subvaults"
#                     f", but at least one subvault already has a file that is"
#                     f" supposed to be copied from the vault"
#                     f". No files have been copied."
#                     f"subvault: {subvault}"
#                     f"file: {file}")
    
#     # TODO: copy and backup files

#     return